In [17]:
import argparse
import sys
import itertools

import numpy as np

import os
import sys

import copy

# currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
# parentdir = os.path.dirname(currentdir)
# sys.path.insert(0, parentdir)
# os.chdir("../..") 


# from experiments import EXPERIMENTS
# from networktools.network_generators import generator

from pathlib import Path
import pandas as pd

In [2]:
outputs_wiki_rhoinits = Path("../../outputs/LtdReal/outputs/wiki2-rhoinits-triads")
outputs_wiki_rhoinits2 = Path("../../outputs/LtdReal/outputs/wiki2-rhoinits-triads-s2000")
outputs_epi_rhoinits = Path("../../outputs/LtdReal/outputs/epinions-rhoinits-triads")
outputs_sam_rhoinits = Path("../../outputs/LtdReal/outputs/sampson-rhoinits-triads")

# Run experiments to compare them later

In [3]:
import os, ast

def process_data(data, n_links, n_triads, k=1):
    for column in data.columns[k:-2]:
        data.loc[:, column] = data[column].str.split(',').map(lambda x: [int(float(x_el))/n_triads for x_el in x])
    data.loc[:, 'rho'] = data['rho'].str.split(',').map(lambda x: [int(float(x_el))/n_links for x_el in x])
    return data

def means_of_data(data, start, steps):
    df = pd.DataFrame(columns=data.columns)
    for _, row in data.iterrows():
        df = df.append(pd.DataFrame({
        'prob': row.p,
        'rho': row.rho[start:][::steps],
        'n0': row.n0[start:][::steps],
        'n1': row.n1[start:][::steps],
        'n2': row.n2[start:][::steps],
        'n3': row.n3[start:][::steps]
        }))
    df = df.groupby('prob').mean()
    return df

def get_rho_init(filename):
    with open(filename) as f:
        first_line = f.readline()
        spl = first_line.split("--rho-init', '")
        if len(spl) == 1:
            return 0.5
        else:
            strval = spl.split("'")[0]
            return float(strval)
        
def read_params(filename):
    # print("Started " + filename)
    params_d = {}
    with open(filename) as f:
        first_line = f.readline()
        # firstline = "# Commit: 126356fd51d619772e0c4137e13dfe116d309ebe# Arguments: ['main.py', 'LtdStatus', '-n', '100', '-p', '0.88', '-q', '0.5', '-ps', '0.25', '-s', '100', '-r', '1', '--rho-init', '0.9', '--ltd-agent-based', '--on-triad-status', 'outputs/test']"
        params = first_line.split("Arguments: ")[1]
        # params.strip('][').strip("'").split(', ')
        try:
            params = ast.literal_eval(params)
            isvalue = False
            last_key = ""
            for par in params:
                if par.startswith('--'):
                    if isvalue:
                        isvalue = False
                        # params_d[last_key] = True
                if not isvalue:
                    if par.startswith('--'):
                        par2 = par.strip("-")
                        par2 = par2.replace('-', '_')
                        params_d[par2] = True
                        isvalue = True
                        last_key = par2
                    elif par.startswith('-'):
                        par2 = par.strip("-")
                        par2 = par2.replace('-', '_')
                        params_d[par2] = np.NAN
                        isvalue = True
                        last_key = par2
                else:
                    isvalue = False
                    try:
                        if "." in par:
                            params_d[par2] = float(par)
                        else:
                            params_d[par2] = int(par)
                    except ValueError:
                        pass
        except (SyntaxError, ValueError):
            # print(params)
            # print(params == "Namespace(command=None, dataset='../data/wikielections/wikielections_triads2.h5', network=<networktools.network_generators.real_graphs.GeneralRealNetwork object at 0x7f1f70ffcfa0>, is_directed=True, agent_based=True, ltd_agent_based=True, on_triad_status=True, save_pk_distribution=False, build_triad='choose_agents', exp_name='outputs/wiki2-s1000-triads', probability=[0.8], q=[0.0], psprob=[0.1], rho_init=[0.0, 0.1, 0.2, 0.30000000000000004, 0.4, 0.5, 0.6000000000000001, 0.7000000000000001], steps=1000, saving_multiplier=10, repetitions=1, no_triad_stats=False, keep_rho_at=[0.67, 0.9, 0.65, 0.92])")
            # print(ord(params[-1]))
            params = params.split("Namespace(")[1]
            params = params[:-1]
            if params[-1] == ")":
                params = params[:-1]
            params = params.split(',')
            params = list(map(lambda x: x.strip(), params))
            """Following is needed in the case some parameters are given as lists."""
            i = len(params) - 1
            while i >= 1:
                if '=' not in params[i]:
                    params[i-1] = ','.join(params[i-1:i+1])
                    params.pop(i)
                i -= 1
            params = list(map(lambda x: x.split('='), params))
            keys = [param[0] for param in params]
            vals = [param[1] for param in params]
            for key, val in zip(keys, vals):
                if key == "steps":
                    key = "s"
                elif key == "n_agents":
                    key = "n"
                if val.startswith("'"):
                    params_d[key] = val
                    continue
                if val.startswith("["):
                    val = ast.literal_eval(val)
                    if len(val) == 1:
                        val = round(val[0],10)
                    else:
                        val = [round(v,10) for v in val]
                    # val = val[1:-1]
                    params_d[key] = val
                else:
                    try:
                        val = ast.literal_eval(val)
                        if val:
                            val = round(val,10)
                            params_d[key] = val
                    except SyntaxError:     
                        try:
                            params_d[key] = int(val)
                        except ValueError:
                            try:
                                if val:
                                    # print(val)
                                    val = round(val,10)
                                    params_d[key] = float(val)
                            except (ValueError, TypeError):
                                pass
    
    if "dataset" in params_d:
        if "wikielections" in params_d["dataset"].lower():
            if "wikielections_triads2" in params_d["dataset"].lower():
                params_d["triads"] = 745129
                params_d["links"] = 94933
            else:
                params_d["triads"] = 747589
                params_d["links"] = 95152
            params_d["dataset"] = "wikielections"
        elif "slashdot" in params_d["dataset"].lower():
            params_d["dataset"] = "slashdot"
            params_d["triads"] = 1251925
            params_d["links"] = 285003
        elif "epinions" in params_d["dataset"].lower():
            params_d["dataset"] = "epinions"
            params_d["triads"] = 10961993
            params_d["links"] = 667110
        elif "sampson" in params_d["dataset"].lower():
            params_d["dataset"] = "sampson"
            params_d["triads"] = 1158
            params_d["links"] = 184
            
    return params_d

def process_folder(directory, no_triad_stats = True, df_files = None, 
                   use_only_files = None, start_index = 0, max_num_rows = 10000):
    """ Processes the whole folder looking for files. 
    If only new files are to be processes then df_files should be given. 
    
    Returns two dataframes: data and information about processes files, 
        and last row of the file list that was processed. 
    """
    if use_only_files is not None:
        files = use_only_files
    else:
        files = [os.path.join(directory, dir, "outputs.tsv") for dir in os.listdir(directory) if os.path.isdir(os.path.join(directory, dir)) ]
    
    # get mod time dataframe
    m_times = [os.stat(file).st_mtime for file in files]
    df_files_all = pd.DataFrame({"File": files, "Modification time": m_times})
    df_files_all.set_index('File', inplace=True)
    
    df_cols = ['q','p', 'ps', 'rho_init', 'steps', 'cur_steps', 'rho']
    usecols = ['q','p', 'ps', 'rho_init', 'rho']
    # usecols_backup = ['q','p', 'ps', 'tr0', 'tr1']
    
    if not no_triad_stats:
    #     usecols = ['q','p', 'ps', 'rho', 'bp']
    #     usecols_backup = ['q','p', 'ps', 'tr0', 'tr1']
    # else:
        for i in range(8):
            df_cols.append('tr' + str(i))
            usecols.append('tr' + str(i))
    
    df = pd.DataFrame({col: [] for col in df_cols})
    last_file_processed = start_index
    # usecols = ['q','p', 'ps', 'rho', 'bp']
    # usecols_backup = ['q','p', 'ps', 'tr0', 'tr1']
    
    empty_folds = np.ones(len(files))
    reps = np.zeros(len(files), dtype = int)
    
    for i, file in enumerate(files[start_index:]):
        if len(df.index) >= max_num_rows:
            break
        
        if (df_files is not None) and (file in df_files.index):
            # checking if file was already processes and there was no more recent version
            if df_files.loc[file].values[0] == df_files_all.loc[file].values[0]:
                continue
        
        inilen = len(df)
        df2 = df.append(pd.read_csv(file,sep = '\t', comment='#', usecols = usecols))
        
        if type(np.array(df2.rho)[-1]) != str:
            if np.isnan(np.array(df2.rho)[-1]):
                """There is most likely error in columns. rho values are in another column"""
                df3 = pd.read_csv(file,sep = '\t', comment='#', usecols = usecols_backup)
                # df3.append(pd.read_csv(file,sep = '\t', comment='#', usecols = usecols_backup))
                df3_len = len(df3)
                df2_len = inilen+df3_len
                # print(df3)
                # print(df3_len)
                # print(df3.tr0.iloc[-df3_len:-1])
                # print(df3.tr0)
                # return df3, df2
                df2.loc[np.array(range(0,df2_len)) >= df2_len - df3_len, 'rho'] = df3["tr0"]
                # df2.loc[np.array(range(0,df2_len)) >= df2_len - df3_len, 'bp'] = df3["tr1"]
                # print(df2)
                # for i, row in enumerate(df3.iterrows()):
                #     df2.rho.iloc[-i] = df3.tr0.iloc[-i]
                #     df2.bp.iloc[-i] = df3.tr1.iloc[-i]
        df = df2
        
        endlen = len(df)
        
        reps[i] = endlen - inilen
        
        if inilen == endlen:
            empty_folds[i] = False
        
        last_file_processed = start_index + i
    
    dicts = [read_params(file) for i, file in enumerate(files)]# if empty_folds[i] == 1]
    
    # rho_inits_one = [round(d["rho_init"], 6) if "rho_init" in d else 0.5 for d in dicts]
    steps_one = [d["s"] for d in dicts]
    
    # rho_inits = [rho_init for rho_init, rep in zip(rho_inits_one, reps) for _ in range(0,rep)]
    steps = [step for step, rep in zip(steps_one, reps) for _ in range(0,rep)]
    # print(df)
    # return df, rho_inits
    # df.rho_init = rho_inits
    df.steps = steps
    
    ms_one = [d["saving_multiplier"] if "saving_multiplier" in d else 1 for d in dicts]
    ms = [m for m, rep in zip(ms_one, reps) for _ in range(0,rep)]

    cur_steps = [[*list(range(0,step, m)), step] for m, step in zip(ms, steps)]
    
    df.cur_steps = cur_steps
    
    df = df.dropna().reset_index()
    
    if "n" in dicts[0]:
        Ls_one = [d["n"]*(d["n"] - 1) for d in dicts]
    elif "dataset" in dicts[0]:
        Ls_one = [d["links"] for d in dicts]
    else:
        raise ValueError("Neither number of nodes nor dataset was given.", dicts[0])
    Ls = [L for L, rep in zip(Ls_one, reps) for _ in range(0,rep)]
    df.rho = np.array(df.rho.str.split(',').map(lambda x: [float(x_el) for x_el in x]))
    df["Lplus"] = df.rho
    df.rho = [np.array(rho) / L for rho, L in zip(df.rho, Ls)]
    
    possibly_shorter = [d["keep_rho_at"][:2] != [0.,1.] if "keep_rho_at" in d else False  for d in dicts]
    possibly_shorters = [p for p, rep in zip(possibly_shorter, reps) for _ in range(0,rep)]
    
    """Correcting number of cur_steps for old simulations"""
    for ind, row in df.iterrows():
        if possibly_shorters[ind]:
            if len(row.rho) != len(row.cur_steps):
                df.at[ind, "cur_steps"] = row.cur_steps[0:len(row.rho)]
                # row.cur_steps = 
        else:
            if len(row.rho) != len(row.cur_steps):
                df.iloc[ind].cur_steps.pop()
            if len(row.rho) != len(row.cur_steps):
                raise ValueError("Wrong lengths")
    
    if not no_triad_stats:
        if "n" in dicts[0]:
            Ts_one = [d["n"]*(d["n"] - 1)*(d["n"] - 2) for d in dicts]
        elif "dataset" in dicts[0]:
            Ts_one = [d["triads"] for d in dicts]
        else:
            raise ValueError("Neither number of nodes nor dataset was given.", dicts[0])
        Ts = [T for T, rep in zip(Ts_one, reps) for _ in range(0,rep)]
        for i in range(8):
            col = 'tr' + str(i)
            df[col] = np.array(df[col].str.split(',').map(lambda x: [float(x_el) for x_el in x]))
            df[col] = [np.array(Ni) / T for Ni, T in zip(df[col], Ts)]
    
    return df, df_files_all, last_file_processed

def group_results(df, cols):
    """group results according to (q,p,ps,rho_init,steps) anc check the outcome

    Args:
        df (_type_): _description_
    """
    mylist = dict()
    
    for group in df.groupby(cols):
        (group_label, df_temp) = group
        q = df.q[0]
        ps = df.ps[0]
        
        rho_crit = get_rho_crit(q, ps)
        
        reps = len(df_temp.rho)
        paradise = np.sum([rho[-1] == 1. for rho in df_temp.rho])
        quasi_stat = np.sum([rho[-1] < rho_crit for rho in df_temp.rho])
        other = np.array(reps) - paradise - quasi_stat
        
        paradise_ratio = paradise / (paradise + quasi_stat)
        
        d = dict(zip(cols, group_label))
        
        d.update({"repetitions": reps, "paradise": paradise, "quasi_stat": quasi_stat, "other": other, "paradise_ratio": paradise_ratio})
        mylist.update({group_label: d})
    
    return pd.DataFrame(data = mylist.values(), index = mylist.keys(), columns = [*cols, 'repetitions', 'paradise', 'quasi_stat', 'other', 'paradise_ratio'])
    

In [4]:
# df, rho_inits = process_folder(outputs_dir_test)
df_rho = process_folder(outputs_sam_rhoinits, no_triad_stats=False)
print(len(df_rho))
df_erho_copy = df_rho.copy(deep=True)
# df_wrho = df_wrho[df_wrho.rho_init >= 0]
# df_wrho = df_wrho.reset_index(drop=True)
df_rho

784


,index,q,p,ps,rho_init,steps,cur_steps,rho,tr0,tr1,tr2,tr3,tr4,tr5,tr6,tr7,Lplus
0,0,0.1,0.8,0.0,0.9,1000,"[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 1...","[0.9003642577685839, 0.910299650732263, 0.9173...","[0.0010117685716456851, 0.0009675247922526496,...","[0.009039414639290502, 0.008582654632237039, 0...","[0.009028832621951135, 0.0081996038494095, 0.0...","[0.0807593108297004, 0.07300570252143018, 0.06...","[0.009015787548851746, 0.008655086716439246, 0...","[0.08133621322327063, 0.07206362930536446, 0.0...","[0.08139943165444459, 0.07265084004341181, 0.0...","[0.7284092409108454, 0.7558749581394552, 0.776...","[600642.0, 607270.0, 611991.0, 615978.0, 61863..."
1,1,0.1,0.8,0.0,0.8,1000,"[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 1...","[0.799317953560882, 0.8260466789584926, 0.8410...","[0.008099804478984798, 0.006178347313303338, 0...","[0.032280717566595785, 0.027615781181396484, 0...","[0.032125545053714226, 0.026947563276130536, 0...","[0.12823379836130164, 0.11731087585989154, 0.1...","[0.03215136152705078, 0.02756597272047154, 0.0...","[0.12826025340465005, 0.11609595080018752, 0.1...","[0.12801805292158094, 0.11601567342726819, 0.1...","[0.5108304666861218, 0.5622698354213509, 0.596...","[533233.0, 551064.0, 561079.0, 569388.0, 57481..."
2,2,0.1,0.8,0.0,0.6,1000,"[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 1...","[0.600610094287299, 0.6918694068444484, 0.7276...","[0.06386931646462464, 0.035355888295130276, 0....","[0.09626972029630014, 0.07338163780983988, 0.0...","[0.09591066150106098, 0.07139413426007479, 0.0...","[0.14424101529712707, 0.14480998117769278, 0.1...","[0.09552441786817416, 0.07260167015249873, 0.0...","[0.14393185618710028, 0.14318281356319057, 0.1...","[0.14409587745586044, 0.1421484213682676, 0.13...","[0.21615713492975228, 0.3171254533733054, 0.37...","[400673.0, 461553.0, 485404.0, 502807.0, 51534..."
3,3,0.1,0.8,0.0,0.4,1000,"[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 1...","[0.3997736505224026, 0.5924765031254215, 0.650...","[0.21809738429863987, 0.08556847281329226, 0.0...","[0.1441482401968328, 0.11075914753822594, 0.08...","[0.14412443065781924, 0.10501566640299807, 0.0...","[0.09541832402191827, 0.13507954256128424, 0.1...","[0.14439518434284715, 0.10773150466343119, 0.0...","[0.09516809580155726, 0.13299962880837454, 0.1...","[0.09548318449026559, 0.13210845874468266, 0.1...","[0.06316515619011981, 0.1907375784677111, 0.27...","[266693.0, 395247.0, 434017.0, 456977.0, 47384..."
4,4,0.1,0.8,0.0,0.2,1000,"[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 1...","[0.1995982671523437, 0.5168068234623975, 0.590...","[0.5135383684335504, 0.1380434196591806, 0.077...","[0.12823480182846314, 0.13290010311081205, 0.1...","[0.12795082062176102, 0.11982811884663674, 0.0...","[0.03193926505882644, 0.11966282043785287, 0.1...","[0.1270668572767744, 0.12575195039807086, 0.10...","[0.03170290293015148, 0.11463800423882774, 0.1...","[0.03164470183478497, 0.1167305981676872, 0.13...","[0.007922282015688205, 0.13244498514093195, 0....","[133154.0, 344767.0, 394107.0, 422460.0, 44352..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
779,1,0.3,0.8,0.6,0.0,1000,"[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 1...","[0.0, 0.4556549894320277, 0.5557928977230142, ...","[1.0, 0.2174838097415315, 0.10967065934086986,...","[0.0, 0.1507743163127362, 0.12390411123232792,...","[0.0, 0.12114859040687218, 0.09655315415727779...","[0.0, 0.09780164975474806, 0.12682091659792155...","[0.0, 0.1348511169456138, 0.11259330306085764,...","[0.0, 0.08652304375673292, 0.11684554076982169...","[0.0, 0.09975804582250691, 0.1266720385608712,...","[0.0, 0.09165942725925842, 0.18694027628005236...","[0.0, 303972.0, 370775.0, 411697.0, 439450.0, ..."
780,2,0.3,0.8,0.6,0.2,1000,"[0, 10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 1...","[0.1998171216141266, 0.5263479785942349, 0.612...","[0.5151469262934213, 0.14269978096136351, 0.07...","[0.12744096807943592, 0.13453602825690547, 0.1...","[0.12761201361832653, 0.117

In [4]:
sim_sets = [[[0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8], list(np.arange(0, 1.01, 0.05)), list(np.arange(0, 1.01, 0.05))], 
            # [1.0, list(np.arange(0,1.01, 0.1)), 0.5], [0.0, 0.5, list(np.arange(0,1.01, 0.1))]
                ]

rho_inits = [0, 0.1, 0.2, 0.3,  0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
steps = 100
saving_multiplier = 1
reps = 10
control_initial_rho = True
keep_rho_at = [0.35, 0.67, 0.3, 0.72]

In [5]:
def explode_sims(sim_sets):
    """ Returns simulation sets in the form of single list. 
    The input are sets with nested lists. 
    """
    exp_sets = []
    
    for sim_set in sim_sets:
        if len(sim_set) == 3:
            q, ps, ph = sim_set
            rho_init = []
        elif len(sim_set) == 4:
            q, ps, ph, rho_init = sim_set
        if not isinstance(q, (list,np.ndarray)):
            q = [q]
        if not isinstance(ph, (list,np.ndarray)):
            ph = [ph]
        if not isinstance(ps, (list,np.ndarray)):
            ps = [ps]
        if not isinstance(rho_init, (list,np.ndarray)):
            rho_init = [rho_init]
        
        for q_ in q:
            for ps_ in ps:
                for ph_ in ph:
                    if len(rho_init) > 0:
                        for rho_init_ in rho_init:
                            exp_sets.append([q_, ps_, ph_, rho_init_])
                    else:
                        exp_sets.append([q_, ps_, ph_])
    
    return exp_sets

def prepare_args2(dataset, network, single_sim_set, 
                  rho_init, steps, saving_multiplier, reps, no_triad_stats, 
                  foldername, keep_rho_at = [], control_initial_rho = False):
    # parser = argparse.ArgumentParser()
    # subparsers = parser.add_subparsers(dest='command', help='Experiments')
    args = argparse.Namespace()
    
    args.dataset = dataset  
    args.network = network
    
    args.is_directed = True
    args.agent_based = True
    args.ltd_agent_based = True
    args.on_triad_status = True
    args.save_pk_distribution = False
    args.build_triad = 'choose_agents'
    
    args.exp_name = foldername
    
    args.probability = [single_sim_set[2]]
    args.q = [single_sim_set[0]]
    args.psprob = [single_sim_set[1]]
    
    if not isinstance(rho_init, (list,np.ndarray)):
        rho_init = [rho_init]
    args.rho_init = rho_init
    args.steps = steps
    args.saving_multiplier = saving_multiplier
    args.repetitions = reps
    
    args.no_triad_stats = no_triad_stats
    args.keep_rho_at = keep_rho_at
    args.control_initial_rho = control_initial_rho
    
    return args

In [6]:
exp_sets = explode_sims(sim_sets)
    
rho_init = rho_inits

all_args = [prepare_args2(0, 0, single_sim_set, rho_init, 
                            steps, saving_multiplier, reps, False, 
                            0, keep_rho_at, control_initial_rho) for single_sim_set in exp_sets]


# Finding what is not finished

In [8]:
all_args[0].rho_init, len(all_args)

# exp_sets

([0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], 5733)

In [11]:
missing_data = []

for args in all_args:
    q = args.q[0]
    p = args.probability[0]
    ps = args.psprob[0]
    rho_inits = args.rho_init
    repetitions = args.repetitions
    steps = args.steps
    saving_multiplier = args.saving_multiplier
    points = steps / saving_multiplier
    
    keep_rho_at = args.keep_rho_at
    
    rho_inits_copy = np.copy(rho_inits)
    rho_inits_copy = rho_inits_copy[::-1]
    largest_first = True
    
    while len(rho_inits_copy) > 0:
        rho_init = rho_inits_copy[0]
        rho_inits_copy = rho_inits_copy[1:]
        inds = np.where( (np.abs(df_erho.q - q) < 1e-6) & (np.abs(df_erho.p - p) < 1e-6) & 
                        (np.abs(df_erho.ps - ps) < 1e-6) & (np.abs(df_erho.rho_init - rho_init) < 1e-6) )
    
        # print(inds)
        # print(rho_init)
        if len(inds[0]) < repetitions:
            # print([q,p,ps])
            missing_data.append([q,ps,p])
            break
        
        triggered_sim_stop = 0
        rho_init_change_order = 0
        correct = 0
        for ind in inds[0]:
            rho = df_erho.loc[ind].rho
            cur_steps = df_erho.loc[ind].cur_steps
            if cur_steps[-1] == steps:
                correct += 1
            else:
                correct += 1
                if largest_first:
                    if rho[-1] > keep_rho_at[3]:
                        rho_init_change_order += 1
                    else:
                        triggered_sim_stop += 1
                else:
                    if rho[-1] < keep_rho_at[2]:
                        rho_init_change_order += 1
                    else:
                        triggered_sim_stop += 1
        if rho_init_change_order == repetitions:
            rho_inits_copy = rho_inits_copy[::-1]
            largest_first = not largest_first
        
        if correct != repetitions:
            missing_data.append([q,ps,p])
            break
        elif triggered_sim_stop == repetitions:
            break
    # print(len(inds[0]))
    # break
    # if len(missing_data) == 1:
    #     print(inds)
    #     print(missing_data[0])
    #     break

In [12]:
len(missing_data)

10

In [15]:
all_args[0]

Namespace(agent_based=True, build_triad='choose_agents', control_initial_rho=True, dataset=0, exp_name=0, is_directed=True, keep_rho_at=[0.35, 0.67, 0.3, 0.72], ltd_agent_based=True, network=0, no_triad_stats=False, on_triad_status=True, probability=[0.0], psprob=[0.0], q=[0.2], repetitions=10, rho_init=[0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9], save_pk_distribution=False, saving_multiplier=1, steps=100)

In [13]:
missing_data

[[0.1, 0.9000000000000001, 0.30000000000000004],
 [0.7000000000000001, 0.30000000000000004, 0.1],
 [0.7000000000000001, 0.30000000000000004, 0.30000000000000004],
 [0.7000000000000001, 0.5000000000000001, 0.7000000000000001],
 [0.7000000000000001, 0.5000000000000001, 0.9000000000000001],
 [0.7000000000000001, 0.7000000000000001, 0.30000000000000004],
 [0.9000000000000001, 0.5000000000000001, 0.1],
 [0.9000000000000001, 0.5000000000000001, 0.30000000000000004],
 [0.9000000000000001, 0.5000000000000001, 0.9000000000000001],
 [0.9000000000000001, 0.7000000000000001, 0.1]]

### In the case of multiple data files

In [14]:
outputs_sam_rhoinits.joinpath("ff")

PosixPath('../../outputs/LtdReal/outputs/sampson-rhoinits-triads/ff')

In [18]:
all_args_copy = copy.deepcopy(all_args)

for file_id in range(1,10):
    filename = outputs_sam_rhoinits.joinpath("df_rho" + str(file_id) + ".h5")
    df_rho = pd.read_hdf(filename, key = "df_rho")
    
    # all_args_copy2 = all_args_copy.copy(deep = True)
    
    data_there = []
    for i, args in enumerate(all_args_copy):
        not_found = False
        
        q = args.q[0]
        p = args.probability[0]
        ps = args.psprob[0]
        rho_inits = args.rho_init
        repetitions = args.repetitions
        steps = args.steps
        saving_multiplier = args.saving_multiplier
        points = steps / saving_multiplier
        
        keep_rho_at = args.keep_rho_at
        
        rho_inits_copy = np.copy(rho_inits)
        rho_inits_copy = rho_inits_copy[::-1]
        largest_first = True
        
        while len(rho_inits_copy) > 0:
            rho_init = rho_inits_copy[0]
            rho_inits_copy = rho_inits_copy[1:]
            inds = np.where( (np.abs(df_rho.q - q) < 1e-6) & (np.abs(df_rho.p - p) < 1e-6) & 
                            (np.abs(df_rho.ps - ps) < 1e-6) & (np.abs(df_rho.rho_init - rho_init) < 1e-6) )
        
            # print(inds)
            # print(rho_init)
            if len(inds[0]) < repetitions:
                # print([q,p,ps])
                # missing_data.append([q,ps,p])
                not_found = True
                break
            
            triggered_sim_stop = 0
            rho_init_change_order = 0
            correct = 0
            for ind in inds[0]:
                rho = df_rho.loc[ind].rho
                cur_steps = df_rho.loc[ind].cur_steps
                if cur_steps[-1] == steps:
                    correct += 1
                else:
                    correct += 1
                    if largest_first:
                        if rho[-1] > keep_rho_at[3]:
                            rho_init_change_order += 1
                        else:
                            triggered_sim_stop += 1
                    else:
                        if rho[-1] < keep_rho_at[2]:
                            rho_init_change_order += 1
                        else:
                            triggered_sim_stop += 1
            if rho_init_change_order == repetitions:
                rho_inits_copy = rho_inits_copy[::-1]
                largest_first = not largest_first
            
            if correct != repetitions:
                # missing_data.append([q,ps,p])
                not_found = True
                break
            elif triggered_sim_stop == repetitions:
                break
        
        if not not_found:
            data_there.append(i)
    
    for index in sorted(data_there, reverse=True):
        del all_args_copy[index]
        

In [21]:
len(all_args_copy), len(all_args)

(4527, 5733)

In [24]:
import pickle

pickle.dump( all_args_copy, open( "sampson_sims.pickle", "wb" ) )

## All files the same

This may become false if the optimization method comes into place, because some sims might not be simulated at all. 

In [8]:


outputs_dir_test_original = Path("outputs/LtdReal/ltd_real_test")
outputs_dir_test_original0 = Path("outputs/LtdReal/ltd_real_test0")
outputs_dir_test_created = Path("outputs/LtdReal/ltd_real_test2")


In [9]:
# df, rho_inits = process_folder(outputs_dir_test)
df_orig = process_folder(outputs_dir_test_original, no_triad_stats=False)
print(len(df_orig))
df_orig.sort_values(by=['q', 'p', 'ps', 'rho_init'], inplace=True)
df_orig.reset_index(inplace=True, drop=True)
df_orig = df_orig.drop(columns=['index'])
df_orig.head()

122


,q,p,ps,rho_init,steps,cur_steps,rho,tr0,tr1,tr2,tr3,tr4,tr5,tr6,tr7,Lplus
0,0.0,0.0,0.0,0.9,10,"[0, 1]","[0.9022177419354839, 0.6491935483870968]","[0.000739247311827957, 0.04536290322580645]","[0.008602150537634409, 0.08897849462365591]","[0.008434139784946237, 0.07688172043010753]","[0.08000672043010752, 0.13958333333333334]","[0.008602150537634409, 0.07600806451612903]","[0.07983870967741935, 0.14045698924731181]","[0.08000672043010752, 0.15255376344086022]","[0.7337701612903226, 0.2801747311827957]","[895.0, 644.0]"
1,0.0,0.0,0.0,0.9,10,"[0, 1]","[0.8941532258064516, 0.6491935483870968]","[0.0013776881720430107, 0.04761424731182796]","[0.009307795698924731, 0.08558467741935484]","[0.010181451612903226, 0.07691532258064517]","[0.08497983870967742, 0.14069220430107526]","[0.00991263440860215, 0.07839381720430108]","[0.0852486559139785, 0.13921370967741936]","[0.084375, 0.14788306451612904]","[0.714616935483871, 0.2837029569892473]","[887.0, 644.0]"
2,0.0,0.0,0.5,0.9,10,"[0, 1]","[0.9022177419354839, 0.6491935483870968]","[0.000739247311827957, 0.04536290322580645]","[0.008602150537634409, 0.08897849462365591]","[0.008434139784946237, 0.07688172043010753]","[0.08000672043010752, 0.13958333333333334]","[0.008602150537634409, 0.07600806451612903]","[0.07983870967741935, 0.14045698924731181]","[0.08000672043010752, 0.15255376344086022]","[0.7337701612903226, 0.2801747311827957]","[895.0, 644.0]"
3,0.0,0.0,0.5,0.9,10,"[0, 1]","[0.8941532258064516, 0.6491935483870968]","[0.0013776881720430107, 0.04761424731182796]","[0.009307795698924731, 0.08558467741935484]","[0.010181451612903226, 0.07691532258064517]","[0.08497983870967742, 0.14069220430107526]","[0.00991263440860215, 0.07839381720430108]","[0.0852486559139785, 0.13921370967741936]","[0.084375, 0.14788306451612904]","[0.714616935483871, 0.2837029569892473]","[887.0, 644.0]"
4,0.0,0.0,1.0,0.9,10,"[0, 1]","[0.9022177419354839, 0.6491935483870968]","[0.000739247311827957, 0.04536290322580645]","[0.008602150537634409, 0.08897849462365591]","[0.008434139784946237, 0.07688172043010753]","[0.08000672043010752, 0.13958333333333334]","[0.008602150537634409, 0.07600806451612903]","[0.07983870967741935, 0.14045698924731181]","[0.08000672043010752, 0.15255376344086022]","[0.7337701612903226, 0.2801747311827957]","[895.0, 644.0]"


In [10]:
# df, rho_inits = process_folder(outputs_dir_test)
df_orig0 = process_folder(outputs_dir_test_original0, no_triad_stats=False)
print(len(df_orig0))
df_orig0.sort_values(by=['q', 'p', 'ps', 'rho_init'], inplace=True)
df_orig0.reset_index(inplace=True, drop=True)
df_orig0 = df_orig0.drop(columns=['index'])
df_orig0.head()

116


,q,p,ps,rho_init,steps,cur_steps,rho,tr0,tr1,tr2,tr3,tr4,tr5,tr6,tr7,Lplus
0,0.0,0.0,0.0,0.9,10,"[0, 1]","[0.9022177419354839, 0.6491935483870968]","[0.000739247311827957, 0.04536290322580645]","[0.008602150537634409, 0.08897849462365591]","[0.008434139784946237, 0.07688172043010753]","[0.08000672043010752, 0.13958333333333334]","[0.008602150537634409, 0.07600806451612903]","[0.07983870967741935, 0.14045698924731181]","[0.08000672043010752, 0.15255376344086022]","[0.7337701612903226, 0.2801747311827957]","[895.0, 644.0]"
1,0.0,0.0,0.0,0.9,10,"[0, 1]","[0.8941532258064516, 0.6491935483870968]","[0.0013776881720430107, 0.04761424731182796]","[0.009307795698924731, 0.08558467741935484]","[0.010181451612903226, 0.07691532258064517]","[0.08497983870967742, 0.14069220430107526]","[0.00991263440860215, 0.07839381720430108]","[0.0852486559139785, 0.13921370967741936]","[0.084375, 0.14788306451612904]","[0.714616935483871, 0.2837029569892473]","[887.0, 644.0]"
2,0.0,0.0,0.5,0.9,10,"[0, 1]","[0.8860887096774194, 0.6491935483870968]","[0.0010080645161290322, 0.04220430107526882]","[0.011223118279569892, 0.08272849462365592]","[0.011693548387096775, 0.07775537634408602]","[0.08998655913978494, 0.14811827956989249]","[0.011155913978494624, 0.08346774193548387]","[0.0905241935483871, 0.1424059139784946]","[0.09005376344086022, 0.1473790322580645]","[0.6943548387096774, 0.27594086021505376]","[879.0, 644.0]"
3,0.0,0.0,0.5,0.9,10,"[0, 1]","[0.9042338709677419, 0.6491935483870968]","[0.0009744623655913978, 0.04576612903225807]","[0.00823252688172043, 0.08810483870967742]","[0.008602150537634409, 0.07731854838709677]","[0.07795698924731183, 0.13961693548387097]","[0.008568548387096774, 0.07708333333333334]","[0.07799059139784946, 0.13985215053763442]","[0.07762096774193548, 0.15063844086021505]","[0.7400537634408603, 0.281619623655914]","[897.0, 644.0]"
4,0.0,0.0,1.0,0.9,10,"[0, 1]","[0.8891129032258065, 0.6491935483870968]","[0.0012768817204301076, 0.04254032258064516]","[0.01129032258064516, 0.08373655913978495]","[0.010752688172043012, 0.08158602150537635]","[0.08756720430107527, 0.14294354838709677]","[0.010080645161290322, 0.0773521505376344]","[0.08823924731182796, 0.14717741935483872]","[0.0887768817204301, 0.1493279569892473]","[0.7020161290322581, 0.27533602150537634]","[882.0, 644.0]"


In [11]:
i = 15
mul = 5
df_orig.loc[mul*i:mul*(i+1)]

,q,p,ps,rho_init,steps,cur_steps,rho,tr0,tr1,tr2,tr3,tr4,tr5,tr6,tr7,Lplus
75,1.0,0.0,0.0,0.9,10,"[0, 1, 2]","[0.9143145161290323, 0.8014112903225806, 0.649...","[0.000571236559139785, 0.009475806451612903, 0...","[0.006149193548387097, 0.038306451612903226, 0...","[0.006216397849462365, 0.02674731182795699, 0....","[0.0727486559139785, 0.12405913978494623, 0.13...","[0.007157258064516129, 0.03400537634408602, 0....","[0.07180779569892473, 0.11680107526881721, 0.1...","[0.07174059139784947, 0.12836021505376344, 0.1...","[0.7636088709677419, 0.522244623655914, 0.2909...","[907.0, 795.0, 644.0]"
76,1.0,0.0,0.5,0.3,10,"[0, 1, 2, 3, 4, 5, 6, 7, 8]","[0.30443548387096775, 0.2872983870967742, 0.28...","[0.33521505376344085, 0.3675403225806452, 0.37...","[0.14798387096774193, 0.14650537634408603, 0.1...","[0.14848790322580646, 0.14049059139784947, 0.1...","[0.063877688172043, 0.05816532258064516, 0.053...","[0.14798387096774193, 0.1411962365591398, 0.13...","[0.06438172043010752, 0.057459677419354836, 0....","[0.063877688172043, 0.0634744623655914, 0.0690...","[0.028192204301075267, 0.025168010752688173, 0...","[302.0, 285.0, 287.0, 286.0, 289.0, 299.0, 285..."
77,1.0,0.0,0.5,0.3,10,"[0, 1, 2, 3, 4, 5]","[0.29939516129032256, 0.3286290322580645, 0.33...","[0.3461693548387097, 0.3117271505376344, 0.305...","[0.14663978494623656, 0.15433467741935483, 0.1...","[0.14670698924731182, 0.13676075268817203, 0.1...","[0.061088709677419355, 0.06854838709677419, 0....","[0.1448252688172043, 0.1393481182795699, 0.135...","[0.06297043010752688, 0.06596102150537635, 0.0...","[0.06290322580645161, 0.08353494623655915, 0.0...","[0.028696236559139787, 0.03978494623655914, 0....","[297.0, 326.0, 334.0, 316.0, 301.0, 288.0]"
78,1.0,0.0,0.5,0.6,10,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[0.5887096774193549, 0.5806451612903226, 0.589...","[0.06861559139784947, 0.07752016129032258, 0.0...","[0.0987231182795699, 0.10574596774193548, 0.10...","[0.09862231182795698, 0.09499327956989247, 0.0...","[0.1453293010752688, 0.14109543010752687, 0.14...","[0.09973118279569892, 0.10023521505376344, 0.1...","[0.1442204301075269, 0.13585349462365592, 0.13...","[0.14432123655913978, 0.14660618279569892, 0.1...","[0.20043682795698925, 0.1979502688172043, 0.20...","[584.0, 576.0, 585.0, 594.0, 607.0, 613.0, 597..."
79,1.0,0.0,0.5,0.6,10,"[0, 1, 2, 3, 4, 5]","[0.5866935483870968, 0.6068548387096774, 0.617...","[0.07066532258064516, 0.06421370967741935, 0.0...","[0.10070564516129032, 0.0929771505376344, 0.09...","[0.10191532258064516, 0.09008736559139785, 0.0...","[0.14002016129032258, 0.14586693548387097, 0.1...","[0.09714381720430107, 0.09459005376344086, 0.0...","[0.14479166666666668, 0.14136424731182795, 0.1...","[0.14358198924731183, 0.14425403225806452, 0.1...","[0.2011760752688172, 0.2266465053763441, 0.243...","[582.0, 602.0, 613.0, 598.0, 591.0, 574.0]"
80,1.0,0.0,0.5,0.9,10,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[0.9022177419354839, 0.8870967741935484, 0.889...","[0.000739247311827957, 0.002217741935483871, 0...","[0.008602150537634409, 0.014986559139784946, 0...","[0.008434139784946237, 0.010483870967741936, 0...","[0.08000672043010752, 0.08521505376344087, 0.0...","[0.008602150537634409, 0.010483870967741936, 0...","[0.07983870967741935, 0.08521505376344087, 0.0...","[0.08000672043010752, 0.08971774193548387, 0.0...","[0.7337701612903226, 0.7016801075268817, 0.706...","[895.0, 880.0, 882.0, 884.0, 878.0, 872.0, 879..."


In [12]:
df_orig0.loc[mul*i:mul*(i+1)]

,q,p,ps,rho_init,steps,cur_steps,rho,tr0,tr1,tr2,tr3,tr4,tr5,tr6,tr7,Lplus
75,1.0,0.0,0.0,0.9,10,"[0, 1, 2]","[0.9122983870967742, 0.8125, 0.6491935483870968]","[0.000739247311827957, 0.007291666666666667, 0...","[0.006182795698924731, 0.033702956989247315, 0...","[0.006518817204301075, 0.027016129032258064, 0...","[0.07426075268817205, 0.11948924731182796, 0.1...","[0.007258064516129033, 0.02886424731182796, 0....","[0.07352150537634408, 0.11764112903225807, 0.1...","[0.07318548387096774, 0.12432795698924731, 0.1...","[0.7583333333333333, 0.5416666666666666, 0.288...","[905.0, 806.0, 644.0]"
76,1.0,0.0,0.5,0.6,10,"[0, 1, 2, 3]","[0.5776209677419355, 0.563508064516129, 0.5403...","[0.07345430107526882, 0.08702956989247312, 0.1...","[0.10349462365591398, 0.11512096774193549, 0.1...","[0.10493951612903225, 0.10036962365591398, 0.1...","[0.14049059139784947, 0.1339717741935484, 0.13...","[0.10248655913978495, 0.10228494623655913, 0.1...","[0.14294354838709677, 0.13205645161290322, 0.1...","[0.1414986559139785, 0.14680779569892474, 0.14...","[0.19069220430107528, 0.18235887096774195, 0.1...","[573.0, 559.0, 536.0, 527.0]"
77,1.0,0.0,0.5,0.6,10,"[0, 1, 2, 3]","[0.6108870967741935, 0.5967741935483871, 0.591...","[0.0572244623655914, 0.06838037634408602, 0.07...","[0.09358198924731183, 0.1022513440860215, 0.10...","[0.09381720430107526, 0.09012096774193548, 0.0...","[0.14448924731182797, 0.1424731182795699, 0.14...","[0.09116263440860214, 0.09539650537634409, 0.0...","[0.1471438172043011, 0.1371975806451613, 0.130...","[0.14690860215053764, 0.1493279569892473, 0.15...","[0.2256720430107527, 0.2148521505376344, 0.211...","[606.0, 592.0, 587.0, 581.0]"
78,1.0,0.0,0.5,0.9,10,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[0.8941532258064516, 0.9002016129032258, 0.892...","[0.0009072580645161291, 0.001075268817204301, ...","[0.009979838709677419, 0.01081989247311828, 0....","[0.010047043010752689, 0.008098118279569893, 0...","[0.08491263440860215, 0.07980510752688172, 0.0...","[0.009979838709677419, 0.008467741935483872, 0...","[0.08497983870967742, 0.07943548387096774, 0.0...","[0.08491263440860215, 0.08215725806451613, 0.0...","[0.7142809139784946, 0.730141129032258, 0.7142...","[887.0, 893.0, 885.0, 881.0, 887.0, 876.0, 866..."
79,1.0,0.0,0.5,0.9,10,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[0.9133064516129032, 0.9173387096774194, 0.928...","[0.0006720430107526882, 0.0008736559139784947,...","[0.00571236559139785, 0.007594086021505376, 0....","[0.007123655913978495, 0.006081989247311828, 0...","[0.07318548387096774, 0.06811155913978495, 0.0...","[0.006317204301075269, 0.006317204301075269, 0...","[0.07399193548387097, 0.0678763440860215, 0.05...","[0.07258064516129033, 0.06938844086021505, 0.0...","[0.7604166666666666, 0.7737567204301076, 0.804...","[906.0, 910.0, 921.0, 913.0, 905.0, 904.0, 903..."
80,1.0,0.0,1.0,0.0,10,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [13]:
# df, rho_inits = process_folder(outputs_dir_test)
df_test = process_folder(outputs_dir_test_created, no_triad_stats=False)
print(len(df_test))
df_test.sort_values(by=['q', 'p', 'ps', 'rho_init'], inplace=True)
df_test.reset_index(inplace=True, drop=True)
df_test = df_test.drop(columns=['index'])
df_test.head()

108


,q,p,ps,rho_init,steps,cur_steps,rho,tr0,tr1,tr2,tr3,tr4,tr5,tr6,tr7,Lplus
0,0.0,0.0,0.0,0.9,10,"[0, 1]","[0.9042338709677419, 0.6491935483870968]","[0.0008736559139784947, 0.04428763440860215]","[0.008736559139784945, 0.08373655913978495]","[0.008299731182795698, 0.07540322580645162]","[0.07785618279569892, 0.1473790322580645]","[0.008064516129032258, 0.08044354838709677]","[0.07809139784946237, 0.14233870967741935]","[0.07852822580645161, 0.15067204301075268]","[0.7395497311827957, 0.275739247311828]","[897.0, 644.0]"
1,0.0,0.0,0.0,0.9,10,"[0, 1]","[0.8830645161290323, 0.6491935483870968]","[0.0015456989247311828, 0.04139784946236559]","[0.0114247311827957, 0.08541666666666667]","[0.0125, 0.08279569892473118]","[0.09146505376344086, 0.1411962365591398]","[0.011760752688172043, 0.07836021505376344]","[0.09220430107526882, 0.14563172043010753]","[0.09112903225806451, 0.148252688172043]","[0.6879704301075269, 0.2769489247311828]","[876.0, 644.0]"
2,0.0,0.0,0.0,0.9,10,"[0, 1]","[0.9022177419354839, 0.6491935483870968]","[0.000739247311827957, 0.04536290322580645]","[0.008602150537634409, 0.08897849462365591]","[0.008434139784946237, 0.07688172043010753]","[0.08000672043010752, 0.13958333333333334]","[0.008602150537634409, 0.07600806451612903]","[0.07983870967741935, 0.14045698924731181]","[0.08000672043010752, 0.15255376344086022]","[0.7337701612903226, 0.2801747311827957]","[895.0, 644.0]"
3,0.0,0.0,0.0,0.9,10,"[0, 1]","[0.8941532258064516, 0.6491935483870968]","[0.0013776881720430107, 0.04761424731182796]","[0.009307795698924731, 0.08558467741935484]","[0.010181451612903226, 0.07691532258064517]","[0.08497983870967742, 0.14069220430107526]","[0.00991263440860215, 0.07839381720430108]","[0.0852486559139785, 0.13921370967741936]","[0.084375, 0.14788306451612904]","[0.714616935483871, 0.2837029569892473]","[887.0, 644.0]"
4,0.0,0.0,0.0,0.9,10,"[0, 1]","[0.9022177419354839, 0.6491935483870968]","[0.000739247311827957, 0.04536290322580645]","[0.008602150537634409, 0.08897849462365591]","[0.008434139784946237, 0.07688172043010753]","[0.08000672043010752, 0.13958333333333334]","[0.008602150537634409, 0.07600806451612903]","[0.07983870967741935, 0.14045698924731181]","[0.08000672043010752, 0.15255376344086022]","[0.7337701612903226, 0.2801747311827957]","[895.0, 644.0]"


In [14]:
assert df_test.equals(df_orig)

AssertionError: 

## All simulations for largest rho_init should be the same. And they should all exist

This should not fail. 

In [15]:
inds_test = np.where(df_test.rho_init == 0.9)

df_test.loc[inds_test].head()

,q,p,ps,rho_init,steps,cur_steps,rho,tr0,tr1,tr2,tr3,tr4,tr5,tr6,tr7,Lplus
0,0.0,0.0,0.0,0.9,10,"[0, 1]","[0.9022177419354839, 0.6491935483870968]","[0.000739247311827957, 0.04536290322580645]","[0.008602150537634409, 0.08897849462365591]","[0.008434139784946237, 0.07688172043010753]","[0.08000672043010752, 0.13958333333333334]","[0.008602150537634409, 0.07600806451612903]","[0.07983870967741935, 0.14045698924731181]","[0.08000672043010752, 0.15255376344086022]","[0.7337701612903226, 0.2801747311827957]","[895.0, 644.0]"
1,0.0,0.0,0.0,0.9,10,"[0, 1]","[0.8941532258064516, 0.6491935483870968]","[0.0013776881720430107, 0.04761424731182796]","[0.009307795698924731, 0.08558467741935484]","[0.010181451612903226, 0.07691532258064517]","[0.08497983870967742, 0.14069220430107526]","[0.00991263440860215, 0.07839381720430108]","[0.0852486559139785, 0.13921370967741936]","[0.084375, 0.14788306451612904]","[0.714616935483871, 0.2837029569892473]","[887.0, 644.0]"
2,0.0,0.0,0.5,0.9,10,"[0, 1]","[0.9022177419354839, 0.6491935483870968]","[0.000739247311827957, 0.04536290322580645]","[0.008602150537634409, 0.08897849462365591]","[0.008434139784946237, 0.07688172043010753]","[0.08000672043010752, 0.13958333333333334]","[0.008602150537634409, 0.07600806451612903]","[0.07983870967741935, 0.14045698924731181]","[0.08000672043010752, 0.15255376344086022]","[0.7337701612903226, 0.2801747311827957]","[895.0, 644.0]"
3,0.0,0.0,0.5,0.9,10,"[0, 1]","[0.8941532258064516, 0.6491935483870968]","[0.0013776881720430107, 0.04761424731182796]","[0.009307795698924731, 0.08558467741935484]","[0.010181451612903226, 0.07691532258064517]","[0.08497983870967742, 0.14069220430107526]","[0.00991263440860215, 0.07839381720430108]","[0.0852486559139785, 0.13921370967741936]","[0.084375, 0.14788306451612904]","[0.714616935483871, 0.2837029569892473]","[887.0, 644.0]"
4,0.0,0.0,1.0,0.9,10,"[0, 1]","[0.9022177419354839, 0.6491935483870968]","[0.000739247311827957, 0.04536290322580645]","[0.008602150537634409, 0.08897849462365591]","[0.008434139784946237, 0.07688172043010753]","[0.08000672043010752, 0.13958333333333334]","[0.008602150537634409, 0.07600806451612903]","[0.07983870967741935, 0.14045698924731181]","[0.08000672043010752, 0.15255376344086022]","[0.7337701612903226, 0.2801747311827957]","[895.0, 644.0]"


In [16]:
inds_orig = np.where(df_orig.rho_init == 0.9)

df_orig.loc[inds_orig].head()

,q,p,ps,rho_init,steps,cur_steps,rho,tr0,tr1,tr2,tr3,tr4,tr5,tr6,tr7,Lplus
0,0.0,0.0,0.0,0.9,10,"[0, 1]","[0.9022177419354839, 0.6491935483870968]","[0.000739247311827957, 0.04536290322580645]","[0.008602150537634409, 0.08897849462365591]","[0.008434139784946237, 0.07688172043010753]","[0.08000672043010752, 0.13958333333333334]","[0.008602150537634409, 0.07600806451612903]","[0.07983870967741935, 0.14045698924731181]","[0.08000672043010752, 0.15255376344086022]","[0.7337701612903226, 0.2801747311827957]","[895.0, 644.0]"
1,0.0,0.0,0.0,0.9,10,"[0, 1]","[0.8941532258064516, 0.6491935483870968]","[0.0013776881720430107, 0.04761424731182796]","[0.009307795698924731, 0.08558467741935484]","[0.010181451612903226, 0.07691532258064517]","[0.08497983870967742, 0.14069220430107526]","[0.00991263440860215, 0.07839381720430108]","[0.0852486559139785, 0.13921370967741936]","[0.084375, 0.14788306451612904]","[0.714616935483871, 0.2837029569892473]","[887.0, 644.0]"
2,0.0,0.0,0.5,0.9,10,"[0, 1]","[0.9022177419354839, 0.6491935483870968]","[0.000739247311827957, 0.04536290322580645]","[0.008602150537634409, 0.08897849462365591]","[0.008434139784946237, 0.07688172043010753]","[0.08000672043010752, 0.13958333333333334]","[0.008602150537634409, 0.07600806451612903]","[0.07983870967741935, 0.14045698924731181]","[0.08000672043010752, 0.15255376344086022]","[0.7337701612903226, 0.2801747311827957]","[895.0, 644.0]"
3,0.0,0.0,0.5,0.9,10,"[0, 1]","[0.8941532258064516, 0.6491935483870968]","[0.0013776881720430107, 0.04761424731182796]","[0.009307795698924731, 0.08558467741935484]","[0.010181451612903226, 0.07691532258064517]","[0.08497983870967742, 0.14069220430107526]","[0.00991263440860215, 0.07839381720430108]","[0.0852486559139785, 0.13921370967741936]","[0.084375, 0.14788306451612904]","[0.714616935483871, 0.2837029569892473]","[887.0, 644.0]"
4,0.0,0.0,1.0,0.9,10,"[0, 1]","[0.9022177419354839, 0.6491935483870968]","[0.000739247311827957, 0.04536290322580645]","[0.008602150537634409, 0.08897849462365591]","[0.008434139784946237, 0.07688172043010753]","[0.08000672043010752, 0.13958333333333334]","[0.008602150537634409, 0.07600806451612903]","[0.07983870967741935, 0.14045698924731181]","[0.08000672043010752, 0.15255376344086022]","[0.7337701612903226, 0.2801747311827957]","[895.0, 644.0]"


In [17]:
df_orig.loc[inds_orig].equals(df_test.loc[inds_test])

False

In [18]:
for ind, row in df_test.loc[inds_test].iterrows():
    q = row.q
    p = row.p
    ps = row.ps
    rho_init = row.rho_init
    
    ind_t = np.where((df_orig.q == q) & (df_orig.p == p) & (df_orig.rho_init == rho_init) & (df_orig.ps == ps))[0]
    
    assert len(ind_t) == 2
    
    any_ = False
    for ind2, row2 in df_orig.loc[ind_t].iterrows():
        if row.equals(row2):
            any_ = True
            break
    assert any_

# New type of simulations. Increasing p or ps may never help!

In [14]:
# all_sims = pd.DataFrame(columns=["q", "p", "ps", "rho_init", "triggered_below_counter", "triggered_above_counter", "reps"])

def create_and_run_one_exp2(args, all_sims):
    q = args.q[0]
    p = args.p[0]
    ps = args.ps[0]
    
    ind_t = np.where((all_sims.q == q) & (all_sims.p == p) & (df_orig.ps < ps))[0]
    
    all_sims.loc[ind_t]
    
    experiment = EXPERIMENTS['LtdReal'](args)
    simulation_history = experiment()
    simulation_history = [sim for sim in simulation_history if sim[-1] == args.repetitions]
    
    
    
    i = len(all_sims)
    for sim in simulation_history:
        all_sims.loc[i] = sim
        i += 1
    # all_sims
    
    ind_t = np.where((df_orig.q == q) & (df_orig.p == p) & (df_orig.rho_init == rho_init) & (df_orig.ps == ps))[0]
    
    pass



In [20]:
ind_t = np.where((all_sims.q == q) & (all_sims.p == p) & (df_orig.ps < ps))[0]

NameError: name 'q' is not defined

In [14]:
args = all_args[0]

In [15]:
all_sims = pd.DataFrame(columns=["q", "p", "ps", "rho_init", "triggered_below_counter", "triggered_above_counter", "reps"])

experiment = EXPERIMENTS['LtdReal'](args)
simulation_history = experiment()

  7%|▋         | 684/9920 [00:00<00:03, 2354.07it/s]


In [16]:
simulation_history

[(0.0, 0.0, 0.0, 0.9, 1, 0, 0),
 (0.0, 0.0, 0.0, 0.9, 2, 0, 1),
 (0.0, 0.0, 0.0, 0.6, 2, 2, 2),
 (0.0, 0.0, 0.0, 0.3, 2, 2, 2),
 (0.0, 0.0, 0.0, 0.0, 2, 2, 2)]

In [17]:
simulation_history = [sim for sim in simulation_history if sim[-1] == reps]
simulation_history

[(0.0, 0.0, 0.0, 0.6, 2, 2, 2),
 (0.0, 0.0, 0.0, 0.3, 2, 2, 2),
 (0.0, 0.0, 0.0, 0.0, 2, 2, 2)]

In [18]:
i = len(all_sims)
for sim in simulation_history:
    all_sims.loc[i] = sim
    i += 1
all_sims

,q,p,ps,rho_init,triggered_below_counter,triggered_above_counter,reps
0,0.0,0.0,0.0,0.6,2.0,2.0,2.0
1,0.0,0.0,0.0,0.3,2.0,2.0,2.0
2,0.0,0.0,0.0,0.0,2.0,2.0,2.0
